Jupiter notebook for checking the influence of the amount of features we select


In [1]:
# Train a GPR model with RDkit featurization
import deepchem as dc
import pandas as pd
import numpy as np
import timeit

from sklearn.feature_selection import RFE
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,RationalQuadratic, Matern
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Error: Unable to import pysam. Please make sure it is installed.
Error: Unable to import pysam. Please make sure it is installed.
Error: Unable to import pysam. Please make sure it is installed.

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
#generate the list of RDKit descriptors
from rdkit.Chem import Descriptors
descriptors = Descriptors._descList

#make a list with the names of descriptors available on RDKit
descriptorlist = []
for item in descriptors:
    descriptorlist.append(item[0])

#function to get the names of the descriptors based on the support vector
def Get_Selected_features(support,descriptorlist):
    for idx, bool in enumerate(support):
        if bool == True:
            print(descriptorlist[idx])

#this is done since RDKit utility in deepchem calculates the descriptors in alphabetical order,
# this way we can later extract the used descriptors
descriptorlist = sorted(descriptorlist)

In [4]:
#load the dataset (polymer smiles and their energy of atomization)

print("loading the data...")
loader1 = dc.data.CSVLoader(["Eat"], feature_field="smiles", featurizer=dc.feat.RDKitDescriptors(descriptorlist))
Data1 = loader1.create_dataset('../../csv_files/Polymers_Eat.csv')

loading the data...


In [5]:
#some RDKit descriptors return nan, make these 0

X1 = np.nan_to_num(Data1.X, copy=True, nan=0.0)
#add data to dataset
Dataset = dc.data.DiskDataset.from_numpy(X=X1, y=Data1.y, w=Data1.w, ids=Data1.ids, tasks = ["Eat"])

In [ ]:
numbers = [5,10,15,20,30,40,50,60,70,100,150,210]
for n_of_feats in numbers:
    print("\n\nnumber of feats:",n_of_feats)
    #select the features using a decisionTreeRegressor
    selector = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=n_of_feats, step = 100)
    X_Selected = selector.fit_transform(Dataset.X, Dataset.y.ravel()) #y is a colum vector but it needs a 1d array -> ravel fixes this
    RFEDataset = dc.data.DiskDataset.from_numpy(X=X_Selected, y=Data1.y, w=Data1.w, ids=Data1.ids, tasks = ["Eat"])

    #find which descriptors are the most important
    selected = selector.support_ #this returns an array with true and false, True are the selected features
    #Get_Selected_features(selected,descriptorlist)
    #Since we repeat our measurement five times, we loop:
    from numpy import average


    r2_scores =[]
    rmse_scores = []
    start = timeit.default_timer()
    for i in range(10):
        #print("\n\nRound:",i)

        
        #split the dataset using the random splitter
        splitter = dc.splits.RandomSplitter()
        train_dataset, test_dataset = splitter.train_test_split(RFEDataset)
        
        # create the GPR model & fit the model
        kernel = 1 * Matern()
        model = dc.models.SklearnModel(GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20))
        #model = dc.models.SklearnModel(RandomForestRegressor(n_estimators=500))
        #print("fitting model...")
        model.fit(train_dataset)
        #print("model is fitted")

        #predict the test set
        predicted = model.predict(test_dataset)

        #calculate r2 score
        metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
        #print('Training set r2_score:', model.evaluate(train_dataset, metric))
        test_score= model.evaluate(test_dataset, metric)
        #print('Test set r2_score:',round(list(test_score.values())[0],3))
        r2_scores.append(list(test_score.values())[0])

        #calculate RMSE score
        from sklearn.metrics import root_mean_squared_error
        RMSE_score = root_mean_squared_error(test_dataset.y,predicted)
        #print('tets set RMSE_score:',round(RMSE_score,3))
        rmse_scores.append(RMSE_score)
    stop = timeit.default_timer()
    print("\naverage r2_score:",average(r2_scores))
    print("average rmse_score:",average(rmse_scores))
    print("Time:",stop-start)



number of feats: 5


c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations


average r2_score: 0.6972479983463399
average rmse_score: 0.2389928680007089
Time: 48.93205089995172


number of feats: 10


c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)



average r2_score: 0.7841616537209084
average rmse_score: 0.41059994434829894
Time: 46.770406499970704


number of feats: 15


...


average r2_score: 0.9681912755474075
average rmse_score: 0.06096310846623215
Time: 73.25689650001004


number of feats: 150


c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations


average r2_score: 0.9732924149693465
average rmse_score: 0.059879534737325865
Time: 79.77480240003206


number of feats: 210


c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\wanne\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations


average r2_score: 0.973957706297948
average rmse_score: 0.056027539891151015
Time: 85.63112529995851
